In [1]:
!nvidia-smi

Sat May 21 04:31:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
!pip install -q transformers
!pip install -q datasets

In [4]:
#transformers
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler 
import pandas as pd
import numpy as np
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments

In [5]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
# model = RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2", num_labels=3)

# Load Dataset


In [6]:
train_dataset_path = "/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/aug_clean_train.csv"
test_dataset_path = "/content/drive/MyDrive/NLP/Setiment_Analysis/en/dataset/covid-19/clean_test.csv"

In [7]:
from datasets import Dataset, load_dataset

dataset = load_dataset('csv', data_files={'train': train_dataset_path,
                                          'val': test_dataset_path})

Using custom data configuration default-e892ee5e9612a018
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/default-e892ee5e9612a018/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/2 [00:00<?, ?it/s]

## Preprocess data

In [8]:
dataset = dataset.rename_column("text_clean", "texts")
dataset = dataset.rename_column("Sentiment", "labels")
# dataset['train'], dataset['validation'] = dataset['train'].train_test_split(0.2).values()

In [9]:
def label_mapping(examples):
    label_to_idx = {'Extremely Negative': 0, 'Negative': 0, 'Neutral': 1,
                    'Positive': 2, 'Extremely Positive': 2}
    examples["labels"] = label_to_idx[examples["labels"]]
    return examples
    
dataset = dataset.map(label_mapping)

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e892ee5e9612a018/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-31d171c191eea004.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e892ee5e9612a018/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-45e6e6fe2b600af6.arrow


In [10]:
def tokenizer_function(examples):
    result = tokenizer(examples["texts"], max_length=128,
                       padding='max_length', truncation=True)
    result["labels"] = examples["labels"]
    return result

tokenized_datasets = dataset.map(tokenizer_function, batched=True, remove_columns=['texts', 'labels'])

Loading cached processed dataset at /root/.cache/huggingface/datasets/csv/default-e892ee5e9612a018/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-a604fc9c7d6eeda2.arrow


  0%|          | 0/4 [00:00<?, ?ba/s]

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 48483
    })
    val: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 3787
    })
})

In [12]:
tokenized_datasets['train'][0]['labels']

1

# Fine-tuning Roberta with Trainer API

In [13]:
from datasets import load_metric

def compute_metrics(eval_preds):
    accuracy_metric = load_metric("accuracy")
    precision_metric = load_metric("precision")
    recall_metric = load_metric("recall")
    f1_metric = load_metric("f1")

    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    precision = precision_metric.compute(predictions=predictions, references=labels, average="macro")
    recall = recall_metric.compute(predictions=predictions, references=labels, average="macro")
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")

    return {
        "accuracy": accuracy['accuracy'],
        "precision": precision['precision'],
        "recall": recall['recall'],
        "f1": f1['f1'],
    }

In [14]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    "/content/output",
    num_train_epochs=20,
    evaluation_strategy='epoch',
    load_best_model_at_end=True,
    metric_for_best_model = 'f1',
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    save_strategy = "epoch",
    logging_steps=200
)

In [15]:
# trainer=Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_datasets['train'],
#     eval_dataset=tokenized_datasets['val'],
#     compute_metrics=compute_metrics,
# )

In [16]:
# trainer.train()

# Hyperparameter search

In [17]:
! pip install -q optuna
! pip install -q ray[tune]

In [18]:
def model_init():
    return RobertaForSequenceClassification.from_pretrained("/content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2", num_labels=3)

In [19]:
from transformers import EarlyStoppingCallback

trainer=Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['val'],
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 2)]
)

loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /content/drive/M

In [20]:
def my_hp_space(trial):
    return {
        "learning_rate": trial.suggest_float("learning_rate", 2e-6, 5e-5, log=True),
        "num_train_epochs": trial.suggest_int("num_train_epochs", 5, 10),
    }

best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize", hp_space=my_hp_space)

[I 2022-05-21 04:32:12,003] A new study created in memory with name: no-name-b8c926f0-54b0-467b-a2b7-1710c05116cc
Trial:
loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_versio

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.401800,0.387745,0.864008,0.865056,0.839258,0.850340
2,0.282600,0.318619,0.890151,0.892633,0.866506,0.877790
3,0.212000,0.325757,0.902033,0.903605,0.881295,0.891129
4,0.152600,0.342127,0.898336,0.889423,0.876773,0.882559
5,0.105800,0.398563,0.896224,0.885068,0.878146,0.881320
6,0.076900,0.476940,0.894904,0.875697,0.877180,0.876417
7,0.047200,0.524188,0.894375,0.873073,0.876737,0.874853


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64


Saving model checkpoint to /content/output/run-0/checkpoint-758
Configuration saved in /content/output/run-0/checkpoint-758/config.json
Model weights saved in /content/output/run-0/checkpoint-758/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-0/checkpoint-1516
Configuration saved in /content/output/run-0/checkpoint-1516/config.json
Model weights saved in /content/output/run-0/checkpoint-1516/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-0/checkpoint-2274
Configuration saved in /content/output/run-0/checkpoint-2274/config.json
Model weights saved in /content/output/run-0/checkpoint-2274/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/run-0/checkpoint-3032
Configuration saved in /content/output/run-0/checkpoint-3032/config.json
M

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.481800,0.439973,0.835490,0.830891,0.811961,0.819970
2,0.347300,0.363453,0.872458,0.867334,0.853394,0.859788
3,0.283500,0.356374,0.881701,0.885242,0.852787,0.866269
4,0.230900,0.346377,0.885661,0.889837,0.858359,0.871522
5,0.193000,0.386147,0.884869,0.882798,0.863620,0.872166
6,0.162500,0.404146,0.880908,0.871826,0.860460,0.865711
7,0.129000,0.443704,0.878268,0.874336,0.850803,0.861059
8,0.111500,0.471183,0.882229,0.879641,0.854853,0.865517
9,0.099100,0.486117,0.883549,0.875567,0.862259,0.868377
10,0.084900,0.502710,0.880116,0.864502,0.859024,0.861668


...

***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
[I 2022-05-21 10:36:01,271] Trial 7 pruned. 
Trial:
loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_vers

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.504400,0.459060,0.825456,0.822690,0.797883,0.807979


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
[I 2022-05-21 10:45:18,762] Trial 8 pruned. 
Trial:
loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_vers

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.539200,0.490431,0.810140,0.805588,0.775810,0.787784


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
[I 2022-05-21 10:54:35,977] Trial 9 pruned. 


In [22]:
best_run

BestRun(run_id='0', objective=3.5190383983523756, hyperparameters={'learning_rate': 3.813488948910996e-05, 'num_train_epochs': 7})

In [24]:
for n, v in best_run.hyperparameters.items():
    setattr(trainer.args, n, v)

trainer.train()

loading configuration file /content/drive/MyDrive/NLP/Setiment_Analysis/en/save_pretrained/exp_2/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file /content/drive/M

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.401800,0.387745,0.864008,0.865056,0.839258,0.850340
2,0.282600,0.318619,0.890151,0.892633,0.866506,0.877790
3,0.212000,0.325757,0.902033,0.903605,0.881295,0.891129
4,0.152600,0.342127,0.898336,0.889423,0.876773,0.882559
5,0.105800,0.398563,0.896224,0.885068,0.878146,0.881320
6,0.076900,0.476940,0.894904,0.875697,0.877180,0.876417
7,0.047200,0.524188,0.894375,0.873073,0.876737,0.874853


***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/checkpoint-758
Configuration saved in /content/output/checkpoint-758/config.json
Model weights saved in /content/output/checkpoint-758/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/checkpoint-1516
Configuration saved in /content/output/checkpoint-1516/config.json
Model weights saved in /content/output/checkpoint-1516/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/checkpoint-2274
Configuration saved in /content/output/checkpoint-2274/config.json
Model weights saved in /content/output/checkpoint-2274/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 3787
  Batch size = 64
Saving model checkpoint to /content/output/checkpoint-3032
Configuration saved in /content/output/checkpoint-3032/config.j

TrainOutput(global_step=5306, training_loss=0.2007515748581525, metrics={'train_runtime': 3928.0897, 'train_samples_per_second': 86.398, 'train_steps_per_second': 1.351, 'total_flos': 2.24123298138432e+16, 'train_loss': 0.2007515748581525, 'epoch': 7.0})